# Live Model Server Testing

Test model server via HTTP calls 

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config spec.image = "mlrun/mlrun"
%nuclio config kind = "job"
%nuclio config spec.maxReplicas = 1

%nuclio: setting spec.image to 'mlrun/mlrun'
%nuclio: setting kind to 'job'


In [3]:
import os
import pandas as pd
import requests
import json
import numpy as np
import mlrun

from datetime import datetime
from mlrun.datastore import DataItem
from mlrun.artifacts import get_model, ChartArtifact

def model_server_tester(context,
                        table: DataItem,
                        addr: str, 
                        label_column: str = "label",
                        model: str = '',
                        match_err: bool = False,
                        rows: int = 20):
    """ Test a model server 
    
    :param table:         csv/parquet table with test data
    :param addr:          function address/url
    :param label_column:  name of the label column in table
    :param model:         tested model name 
    :param match_err:     raise error on validation (require proper test set)
    :param rows:          number of rows to use from test set
    """
        
    table = table.as_df()

    y_list = table.pop(label_column).values.tolist()
    context.logger.info(f'testing with dataset against {addr}, model: {model}')
    if rows and rows < table.shape[0]:
        table = table.sample(rows)
    
    count = err_count = match = 0
    times = []
    for x, y in zip(table.values, y_list):
        count += 1
        event_data = json.dumps({"instances":[x.tolist()]})
        had_err = False
        try:
            start = datetime.now()
            resp = requests.put(f'{addr}/{model}/predict', json=event_data)
            if not resp.ok:
                context.logger.error(f'bad function resp!!\n{resp.text}')
                err_count += 1
                continue
            times.append((datetime.now()-start).microseconds)
                
        except OSError as err:
            context.logger.error(f'error in request, data:{event_data}, error: {err}')
            err_count += 1
            continue
            
        y_resp = resp.json()[0]
        if y == y_resp:
            match += 1
        
    context.log_result('total_tests', count)
    context.log_result('errors', err_count)
    context.log_result('match', match)
    if count - err_count > 0:
        times_arr = np.array(times)
        context.log_result('avg_latency', int(np.mean(times_arr)))
        context.log_result('min_latency', int(np.amin(times_arr)))
        context.log_result('max_latency', int(np.amax(times_arr)))
        
        chart = ChartArtifact('latency', header=['Test', 'Latency (microsec)'])
        for i in range(len(times)):
            chart.add_row([i+1, int(times[i])])
        context.log_artifact(chart)

    context.logger.info(f'run {count} tests, {err_count} errors and {match} match expected value')
    
    if err_count:
        raise ValueError(f'failed on {err_count} tests of {count}')
    
    if match_err and match != count:
        raise ValueError(f'only {match} results match out of {count}')

In [4]:
# nuclio: end-code
# marks the end of a code section

### Deploy model server for testing

In [5]:
project_name = 'sk-project'
MODEL_PATH = 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'

artifact_path = mlrun.set_environment(api_path = 'http://mlrun-api:8080',
                                      artifact_path = os.path.abspath('./'))

# import model server function from hub
fn = mlrun.import_function('hub://model_server')
fn.add_model("mymodel", MODEL_PATH)
address = fn.deploy()

> 2020-10-19 05:10:56,243 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match
> 2020-10-19 05:10:56,564 [info] deploy started
[nuclio] 2020-10-19 05:10:58,696 (info) Build complete
[nuclio] 2020-10-19 05:11:08,070 (info) Function deploy complete
[nuclio] 2020-10-19 05:11:08,078 done updating default-model-server, function address: 192.168.224.209:32324
> 2020-10-19 05:11:08,087 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match


In [6]:
user_name = os.getenv('V3IO_USERNAME')

fn.apply(mlrun.mount_v3io())
fn.set_envs({'SERVING_MODEL_iris_dataset_v1': MODEL_PATH,
             'INFERENCE_STREAM': 'users/{}/tststream'.format(user_name)})

address = fn.deploy(project='sk-project')

> 2020-10-19 05:11:08,123 [info] deploy started
[nuclio] 2020-10-19 05:11:10,233 (info) Build complete
[nuclio] 2020-10-19 05:11:27,590 (info) Function deploy complete
[nuclio] 2020-10-19 05:11:27,598 done updating sk-project-model-server, function address: 192.168.224.209:30499


### Run model server tester locally

In [7]:
# run the function locally
DATA_PATH  = 'https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv'

gen = mlrun.run_local(name='model_server_tester', 
                      handler=model_server_tester, 
                      params={'addr': address, 'model': 'mymodel'},
                      inputs={'table': DATA_PATH},
                      project=project_name, 
                      artifact_path=os.path.join(artifact_path, 'data')) 

> 2020-10-19 05:11:27,645 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match
> 2020-10-19 05:11:27,645 [info] starting run model_server_tester uid=45b795c0213341b3b5db47d14ae03653  -> http://mlrun-api:8080
> 2020-10-19 05:11:27,678 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match
> 2020-10-19 05:11:28,359 [info] testing with dataset against http://192.168.224.209:30499, model: mymodel
> 2020-10-19 05:11:29,224 [info] run 20 tests, 0 errors and 8 match expected value


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...4ae03653,0,Oct 19 05:11:27,completed,model_server_tester,v3io_user=idanbkind=handlerowner=idanbhost=idan-jupyter-5f8568c8b5-cpvv6,table,addr=http://192.168.224.209:30499model=mymodel,total_tests=20errors=0match=8avg_latency=40873min_latency=39586max_latency=43288,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run 45b795c0213341b3b5db47d14ae03653 --project sk-project , !mlrun logs 45b795c0213341b3b5db47d14ae03653 --project sk-project
> 2020-10-19 05:11:29,299 [info] run executed, status=completed


### Save

In [8]:
test_func = mlrun.code_to_function(name='model_server_tester', kind='job')
test_func.spec.default_handler = "model_server_tester"
test_func.spec.description = "test model servers"
test_func.metadata.categories = ["ml", "test"]
test_func.metadata.labels = {"author": "yaronh"}
test_func.export('function.yaml')

> 2020-10-19 05:11:42,786 [info] function spec saved to path: function.yaml


### Run remotely

In [9]:
test_func.run(mlrun.NewTask(name='model_server_tester', 
                            handler=model_server_tester, 
                            params={'addr': address, 'model': 'mymodel'},
                            inputs={'table': DATA_PATH},
                            project=project_name, 
                            artifact_path=os.path.join(artifact_path, 'data')))

> 2020-10-19 05:11:42,805 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match
> 2020-10-19 05:11:42,806 [info] starting run model_server_tester uid=0b5caed929754bc7a2dc2e36e7cc8408  -> http://mlrun-api:8080
> 2020-10-19 05:11:42,927 [info] Job is running in the background, pod: model-server-tester-2gfrc
> 2020-10-19 05:11:47,364 [info] testing with dataset against http://192.168.224.209:30499, model: mymodel
> 2020-10-19 05:11:48,209 [info] run 20 tests, 0 errors and 5 match expected value
> 2020-10-19 05:11:48,235 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...e7cc8408,0,Oct 19 05:11:46,completed,model_server_tester,v3io_user=idanbkind=jobowner=idanbhost=model-server-tester-2gfrc,table,addr=http://192.168.224.209:30499model=mymodel,total_tests=20errors=0match=5avg_latency=39525min_latency=38195max_latency=42526,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run 0b5caed929754bc7a2dc2e36e7cc8408 --project sk-project , !mlrun logs 0b5caed929754bc7a2dc2e36e7cc8408 --project sk-project
> 2020-10-19 05:11:52,118 [info] run executed, status=completed
